<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/ex14notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 ex14notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo15.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の DCT の関数
from scipy.fft import dct, idct

# 科学技術計算のライブラリ SciPy の中の WAVE ファイルを扱うモジュール
from scipy.io.wavfile import read, write

---
## 演習問題 直交展開，離散コサイン変換
---



---
### 直交展開の練習問題

#### 問題1

$a, b, c, d$ を実数として

$$
\mathbf{u}_1 = \left( \frac{1}{2}, \frac{1}{2}, \frac{1}{2}, \frac{1}{2} \right)\quad
\mathbf{u}_2 = \left( \frac{1}{\sqrt{2}}, -\frac{1}{\sqrt{2}}, 0, 0 \right)
\quad
\mathbf{u}_3 = \left( 0, 0, \frac{1}{\sqrt{2}}, -\frac{1}{\sqrt{2}} \right)
\quad
\mathbf{u}_4 = ( a, b, c, d )
$$

とおく．$\{ \mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3, \mathbf{u}_4 \}$ が正規直交基底を成すとき，次の問に答えなさい．ただし，$a>0$ とする．

(1) $a, b, c, d$ の値を求めなさい．

(2) $\mathbf{x} = (6, 2, 2, 12)$ を $\{\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3, \mathbf{u}_4\}$ で展開したときの展開係数を $c_1, c_2, c_3, c_4$ とおくとき，これらの値を求めなさい．ただし，$c_k$ は $\mathbf{u}_k$ に対応する展開係数とする（$k = 1, 2, 3, 4$）．

(3) (2) で求めた展開係数を用いて，$c_1\mathbf{u}_1 + c_2\mathbf{u}_2 + c_3\mathbf{u}_3 + c_4\mathbf{u}_4 = \mathbf{x}$ であることを示しなさい．

---
### 大津市の日平均気温のDCT

#### データの準備


大津市の2004年度の気温（日ごとの平均気温）のデータを読み込んで `temp2004` という変数に代入します．この年はうるう年ですので，値が $366$ 個あります．

In [ ]:
temp2004 = np.loadtxt('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/temp2004.txt')
N_temp2004 = len(temp2004)
temp2004

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(temp2004, '-', color='blue', label='temp2004')
ax.axhline(0, color='gray')
ax.legend()
plt.show()

以下，この気温のデータを表す $N$ 次元ベクトルを $\mathbf{x}$ と表記することにします．
記号をちゃんと定義せずに使っているところがあります．notebookA を参照して補ってね．

#### DCTしてみよう

上記の気温のデータをDCTしてみましょう．

次のセルを実行すると，$N$ 次元のDCTの基底のうち最初の4つ $\mathbf{u}_0, \mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3$ をグラフに描きます．曲線に見えますが，実際にはそれぞれ $N$ 個の点がプロットされています．

In [ ]:
# DCT の基底を求める
U_dct = dct(np.eye(len(temp2004)), norm='ortho')

# 0 番から 3 番の基底ベクトルを可視化
fig, ax = plt.subplots(figsize=(8, 6))
for k in range(4):
    ax.plot(U_dct[:, k], '.', label=f'u{k}')
ax.axhline(0, color='gray')
ax.set_ylim(-0.1, 0.1)
ax.legend()
plt.show()

##### 問題2

次のことを考えて／調べてメモしておこう：

(1) ここでは $N$ はいくつか．基底は，何次元のベクトルが全部で何本から成るか．

(2) 基底ベクトル $\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3$ の周期はそれぞれいくつか．



次のセルを実行すると，`temp2004` にDCTを適用します．`c_temp2004` という配列に展開係数の値 $c_k$ ($k = 0, 1, \ldots, N-1$)が格納されます．

In [ ]:
# temp2004 の DCT
c_temp2004 = dct(temp2004, norm='ortho')
print(c_temp2004)

次のセルを実行すると，`c_temp2004` の値を可視化します．横軸が $k$ で縦軸が $c_k$ です．

In [ ]:
# 展開係数の可視化
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
ax[0].stem(c_temp2004, markerfmt=' ')
ax[0].set_ylim(-330, 330)
ax[1].stem(c_temp2004, markerfmt='o')
ax[1].set_xlim(-0.5,10.5)
ax[1].set_ylim(-330, 330)
plt.show()

次のセルを実行すると，$0$ 以上 $N-1$ 未満の整数 $H$ に対して

$$
\begin{aligned}
\mathbf{z}_\textrm{low} &= c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{H}\mathbf{u}_{H}\\
\mathbf{z}_\textrm{high} &= c_{H+1}\mathbf{u}_{H+1} +  \cdots + c_{N-1}\mathbf{u}_{N-1}
\end{aligned}
$$

を計算し，$\mathbf{x}$ と重ねて表示します．


In [ ]:
H = 0 #@param {type:"number", min:0, max:366}
N = len(c_temp2004)
assert 0 <= H and H < N-1

fig, ax = plt.subplots(1, 2, figsize=(16, 6))

z_low = c_temp2004[:H+1] @ U_dct[:, :H+1].T
ax[0].plot(temp2004, '-', label='temp2004')
ax[0].plot(z_low, '-', color='red', label='z_low', linewidth=2)
ax[0].axhline(0, color='gray')
ax[0].legend()

z_high = c_temp2004[H+1:] @ U_dct[:, H+1:].T
ax[1].plot(temp2004, '-', label='temp2004')
ax[1].plot(z_high, '-', color='red', label='z_high', linewidth=2)
ax[1].axhline(0, color='gray')
ax[1].legend()

plt.show()

$\mathbf{z}_\textrm{low}$ の方は，元のベクトルから長い周期の（周波数の低い）波の成分のみを取り出して表したものとなっており，$\mathbf{z}_\textrm{high}$ の方は逆に，短い周期の（周波数の高い）波の成分のみを取り出して表したものとなっています．

##### 問題3

次のことをやろう／考えよう．

(1) $H = 0, 1, 2, 3$ で実行して結果を観察しよう．もっと大きな $H$ でもやってみよう．

(2) $k = 104$ のとき $\frac{2N}{k} =7.04$ なので，$\mathbf{u}_{104}$ はおよそ1週間周期の変動の成分と言えます．$H=104$ として実行しよう．$\{\mathbf{u}_0, \ldots, \mathbf{u}_{104}\}$ と $\{\mathbf{u}_{105}, \ldots, \mathbf{u}_{366}\}$ のうち，1週間より短い周期での変動を表す成分はどっち？



### 猫の鳴き声のDCT

#### データの準備

ネットから WAVE 形式のファイルを入手します．まずは，WAVE形式のファイルを読み込む関数を定義します．

In [ ]:
# WAVE ファイルを読み込む関数
#
def readWAVE(filename):

    framerate, data = read(filename)

    # チャンネル数とフレーム数（データ点の数）を求める
    if data.ndim == 1:
        nchannels = 1
    else:
        nchannels = data.shape[1]
    nframes = data.shape[0]

    print('filename = ', filename)
    print('nchannels = ', nchannels)       # チャンネル数
    print('framerate = ', framerate)       # 標本化周波数
    print('nframes = ', nframes)             # フレーム数
    print('duration = {0:.2f}[sec]'.format(nframes / framerate))   # 長さ（秒単位）
    print('dtype = ', data.dtype)            # データ型（量子化ビット数に対応）

    assert data.dtype == 'uint8' or data.dtype == 'int16' or data.dtype == 'int32' or data.dtype == 'float32'

    # 最初の10秒分だけ取り出す（元がそれより短ければそのまま）
    nframesNew = min(framerate * 10, nframes)
    if nchannels == 1:
        dataNew = data[:nframesNew]
    else:
        # 多チャンネル信号なら0番目と1番目の平均値を取り出す
        if data.dtype == 'float32':  # 浮動小数点数のときは [0, 1] の値なので普通に足して2で割る
            dataNew = (data[:nframesNew, 0] + data[:nframesNew, 1])/2
        else: # 整数型のときはオーバーフローする可能性があるので，いったん64bit整数にしてから
            data64 = (data[:nframesNew, 0].astype(np.int64) + data[:nframesNew, 1].astype(np.int64))//2
            dataNew = data64.astype(data.dtype)

    return framerate, dataNew

WAVE ファイルを入手します．

In [ ]:
# WAVE ファイルを入手
#
### こちらのサイトの素材を利用させてもらってます http://www.ne.jp/asahi/music/myuu/wave/wave.htm

! wget -nc http://www.ne.jp/asahi/music/myuu/wave/cat1.wav
fnCat = 'cat1.wav'

import os

if not os.path.exists(fnCat):
    print(f'{fnCat}のダウンロードがうまくできていないようです')

In [ ]:
# cat1
fr_cat1, cat1 = readWAVE('cat1.wav')

# cat1 に正弦波を重ねる
t = np.linspace(0, len(cat1)/fr_cat1, num=len(cat1))
vmax = 2**(15-2) # 16bit量子化で最大振幅の 1/4
cat1 = cat1.astype(float) + 0.1*vmax*(np.sin(2*3400*np.pi*t) + np.sin(2*3600*np.pi*t)) # 3400Hz, 3600Hz の正弦波をのせる

N_cat1 = len(cat1)
print(fr_cat1, N_cat1)

データをグラフに描くと次のようになります．2段目のグラフは，1段目のグラフの一部区間を拡大したものです．
横軸はデータ点の番号を表しています．このデータは標本化周波数 44100 [Hz] の音信号なので，横軸の長さ 1 は $1/44100$ [s] に対応しています．データ点の数 $N = 187392$ ですので，$N/44100 = 4.25$ [s] の長さの音です．

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
ax[0].plot(cat1, '-', label='cat1')
ax[0].axhline(0, color='gray')
ax[0].set_xlim(0, len(cat1))
ax[0].set_ylim(-2**15, 2**15)
ax[0].legend()
ax[1].plot(cat1, '.')
ax[1].axhline(0, color='gray', label='cat1')
ax[1].set_xlim(441*50, 441*51)
ax[1].set_ylim(-2**15, 2**15)
ax[1].legend()
plt.tight_layout()
plt.show()

次のセルを指示に従って書き換えて実行すると，上記で作った `cat1` を WAVE 形式のファイルとしてダウンロードできます．音を聞いてみましょう．

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    write('cat1.wav', fr_cat1, cat1.astype('int16'))
    files.download('cat1.wav')

#### DCT してみよう

`cat1` の音を聞いてみると，猫の鳴き声に高音のノイズが加わっています．
DCT の展開係数がどのようになっているか，眺めてみましょう．

In [ ]:
# cat1 の DCT
c_cat1 = dct(cat1, norm='ortho')
print(c_cat1, c_cat1.shape)

In [ ]:
# 展開係数の可視化
fig, ax = plt.subplots(figsize=(8, 6))
ax.stem(c_cat1, markerfmt=' ', label='c_cat1')
kmax = 5000*2*N_cat1/fr_cat1
ax.set_xlim(0, kmax)
#ax.stem(np.arange(len(c_dct))*fr_cat1/(2*N_cat1), c_dct, markerfmt=' ')
#ax.set_xlim(0, 5000)
ax.legend()
plt.show()

上の図の左が DCT の展開係数を可視化したものです．横軸は展開係数の番号 $k$ を表します．
このグラフを見ると，$k = 30000$ の前後に大きなトゲが見えます．実は，これらが高音のノイズの成分です．



これらの成分を取り除くために，$k = 25000$ 以上の成分を無視して元の信号を近似させてみましょう．元の信号 $\mathbf{x}$ は，$N = 187392$ として

$$
\mathbf{x} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{N-1}\mathbf{u}_{N-1}
$$

と表せますが，$k = 24500$ 番以降の成分を捨てて

$$
\mathbf{z} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{2499}\mathbf{u}_{2499}
$$

という信号を作ってみます．

In [ ]:
# 25000 番以降の成分を 0 にして再構成
c_cat2 = np.copy(c_cat1)
c_cat2[25000:] = 0
cat2 = idct(c_cat2, norm='ortho')

In [ ]:
# 展開係数の可視化
fig, ax = plt.subplots(figsize=(8, 6))
ax.stem(c_cat2, markerfmt=' ', label='c_cat2')
ax.set_xlim(0, 50000)
ax.legend()
plt.show()

このようにして得られたデータは `cat2` という変数に格納されています．次のセルを実行するとその波形を `cat1` と比較することができます．横軸を時間軸としてグラフを描いても，`cat1` と `cat2` の違いはそれほど大きくはないようです．

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(cat1, '.', label='cat1')
ax.plot(cat2, '.', label='cat2')
ax.axhline(0, color='gray')
ax.set_xlim(441*50, 441*51)
ax.set_ylim(-2**15, 2**15)
ax.legend()
plt.tight_layout()
plt.show()

しかし，音を聞いてみるとどうでしょうか．次のセルを実行して WAVE ファイルをダウンロードして聞いてみましょう．

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    write('cat2.wav', fr_cat1, cat2.astype('int16'))
    files.download('cat2.wav')

高音のノイズは取り除けていますが，それだけでなく猫の鳴き声も少し変化していることが分かるでしょう．「大きい $k$ に対応する成分」 = 「周期の短い成分」 = 「周波数の高い音の成分」ということで，高い周波数の成分をすべて除いてしまった結果，元の猫の鳴き声の高周波成分も取り除かれてしまった，というわけです．

##### やってみよう

以下のように書かれているコードセルの 3 行目の 25000 を変えると，何番目以降の成分を 0 にするかを変えることができます．適当に変えてからそれ以降のセルを実行して音を聞いてみましょう．
```
# 25000 番以降の成分を 0 にして再構成
c_cat2 = np.copy(c_cat1)
c_cat2[25000:] = 0
cat2 = idct(c_cat2, norm='ortho')
```